In [1]:

from Sfilter import sfilter
import MDAnalysis as mda
import pandas as pd
# Sfilter.sfilter is the low-level API for determining the state of the selectivity filter

/home/chui/Software-2023-04-11/miniconda3/envs/py3.11_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
top = "../test/01-NaK2K/1-Charmm/em.pdb"
xtc = "../test/01-NaK2K/1-Charmm/with_water/fix_atom_c_100ps.xtc"
u = mda.Universe(top, xtc)
sf = sfilter(u)

## Find SF according to the sequence

In [3]:
sf.detect_SF_sequence(SF_seq1="THR VAL GLY TYR GLY".split())  # provide SF_seq2 if analysising TRAAK or Trek2
sf.sf_oxygen  # 6 mda selection. They are the oxygen of the selectivity filter. They ordered from top to bottom

(<AtomGroup with 4 atoms>,
 <AtomGroup with 4 atoms>,
 <AtomGroup with 4 atoms>,
 <AtomGroup with 4 atoms>,
 <AtomGroup with 4 atoms>,
 <AtomGroup with 4 atoms>)

In [4]:
for sf_oxygen in sf.sf_oxygen:
    print(sf_oxygen[0])  # Oxygen on chain A

<Atom 754: O of type O of resname GLY, resid 67 and segid A and altLoc >
<Atom 747: O of type O of resname TYR, resid 66 and segid A and altLoc >
<Atom 726: O of type O of resname GLY, resid 65 and segid A and altLoc >
<Atom 719: O of type O of resname VAL, resid 64 and segid A and altLoc >
<Atom 703: O of type O of resname THR, resid 63 and segid A and altLoc >
<Atom 696: OG1 of type O of resname THR, resid 63 and segid A and altLoc >


In [5]:
for sf_oxygen in sf.sf_oxygen:
    print(sf_oxygen[1])  # Oxygen on chain B

<Atom 2244: O of type O of resname GLY, resid 67 and segid B and altLoc >
<Atom 2237: O of type O of resname TYR, resid 66 and segid B and altLoc >
<Atom 2216: O of type O of resname GLY, resid 65 and segid B and altLoc >
<Atom 2209: O of type O of resname VAL, resid 64 and segid B and altLoc >
<Atom 2193: O of type O of resname THR, resid 63 and segid B and altLoc >
<Atom 2186: OG1 of type O of resname THR, resid 63 and segid B and altLoc >


# What site is each K+ ion in ?
![SF_definition](./SF_definition.jpg)

In [6]:
POT = u.select_atoms("name POT")
POT

<AtomGroup with 160 atoms>

In [7]:
s5_z_cutoff = 3    # A, z cutoff of the SCav
r_cutoff = 3.5     # A, the radius of the SF cylinder (S1, S2, S3, S4)
s0_r_cutoff = 4.0  # A, the radius of the S0

# label each POT atom with the binding site integer
POT_array = sf.state_detect(POT, s5_z_cutoff, r_cutoff, s0_r_cutoff)
print(POT_array.shape)

(160,)


# Which atom does each binding site have?

In [8]:
sf.state_2_list(POT_array, POT)  # state list with the index of atoms in every binding site

[array([], dtype=int64),
 array([5963]),
 array([5962]),
 array([5961]),
 array([5960]),
 array([], dtype=int64)]

# Loop over trajectory
POT is a referrence to the current frame, there is no need to redo the selection. The coordinate inside the POT will be automaticaly updated while looping over the trajectory

In [9]:
site_occu = []
for ts in u.trajectory:
    POT_array = sf.state_detect(POT, s5_z_cutoff, r_cutoff, s0_r_cutoff)    
    site_occu.append(sf.state_2_list(POT_array, POT))


In [10]:
pd.DataFrame(site_occu, columns = ["S0", "S1", "S2", "S3", "S4", "SCav"])

,S0,S1,S2,S3,S4,SCav
0,[],[5963],[5962],[5961],[5960],[]
1,[5963],[5962],[],[5961],[5960],[]
2,[5963],[5962],[],[5961],[5960],[]
3,[],[5963],[5962],[5961],[5960],[]
4,[],[5963],[5962],[5961],[5960],[]
5,[],[5963],[5962],[5961],[5960],[]
6,[5963],[5962],[],[5961],[5960],[]
7,[],[5962],[5961],[],[5960],[]
8,[],[5962],[],[5961],[5960],[]
9,[],[5962],[],[5961],[5960],[]


In [11]:
# Do the same with water
wat = u.select_atoms("resname SOL")
site_occu_wat = []
for ts in u.trajectory:
    wat_array = sf.state_detect(wat, s5_z_cutoff, r_cutoff, s0_r_cutoff)    
    site_occu_wat.append(sf.state_2_list(wat_array, wat))


In [12]:
pd.DataFrame(site_occu_wat, columns = ["S0", "S1", "S2", "S3", "S4", "SCav"])

,S0,S1,S2,S3,S4,SCav
0,"[56425, 60299, 60300, 60301]",[],[],[],[],"[31255, 32579, 32580, 32581, 40145, 40146, 401..."
1,[],[],[],[],[],"[36476, 36477, 39977, 39978, 39979, 41862, 522..."
2,[],[],[],[],[],"[36476, 36477, 36478, 37471, 41862, 47115, 490..."
3,"[35385, 35386, 48137, 48138]",[],[],[],[],"[31295, 31296, 31297, 37469, 37471, 40427, 404..."
4,"[35384, 35385, 35386, 42595, 50423, 50424]",[],[],[],[],"[37016, 37017, 37018, 38117, 38118, 38119, 399..."
5,"[42764, 42765, 42766, 47197, 53675, 53676, 53677]",[],[],[],[],"[30694, 36476, 36477, 36478, 36844, 37470, 384..."
6,"[55301, 55302]",[],[],[],[],"[30692, 30693, 30694, 38414, 38415, 38416, 390..."
7,"[41922, 41923, 43610, 43611, 43612]",[],[],[],[],"[30692, 30693, 30694, 32579, 32580, 32581, 364..."
8,"[37128, 41121, 41122, 54627]",[],[],[],[],"[32580, 39023, 39025, 40088, 40089, 40090, 547..."
9,"[33339, 37620, 41318, 41319, 41320]",[],[],[],[],"[30775, 33674, 33675, 33676, 37121, 37122, 371..."
